# Piperack Segregator 1.0

> Segregate different elements and apply profiles as required with integrated profile applicator

In [3]:
from base import *
from base.piperack.load import *

base_point_of_first_portal = Point3D(0,0,0)
width_of_piperack = 8
portal_distances = [0,8,16,24,32,40,48]
column_distances = [0,width_of_piperack]
braces_placement = [False,True,False,False,True,False]
brace_pattern = BracePattern.X_Pattern
bracket_size = 2
max_expansion_bay_length = 8


tiers = [
    Tier(base=base_point_of_first_portal.shift_y(3),
         loads=[uniform_operating_load.set_force_value(-0.4)],
         wind_loads = [wl_gx.set_fx(1.42),wl_gx_o.set_fx(-1.42)],
         ).set_intermediate_transverse_beam(False),

    Tier(base=base_point_of_first_portal.shift_y(6),
         loads=[uniform_operating_load.set_force_value(-0.4)],
         wind_loads = [wl_gx.set_fx(1.316),wl_gx_o.set_fx(-1.316)],
         ).set_intermediate_transverse_beam(False),

    Tier(base=base_point_of_first_portal.shift_y(9.5),
         loads=[uniform_operating_load.set_force_value(-0.35)],
         wind_loads = [wl_gx.set_fx(1.014),wl_gx_o.set_fx(-1.014)]
         ).set_bracket_provision(True),

    Tier(base=base_point_of_first_portal.shift_y(12),
         loads=[uniform_operating_load.set_force_value(-0.3)],
         wind_loads = [wl_gx.set_fx(1.118),wl_gx_o.set_fx(-1.118)]
         ).set_bracket_provision(True),

    Tier(base=base_point_of_first_portal.shift_y(14.5),
         tier_type= TierType.ElectricalIntrumentation,
         wind_loads = [wl_gx.set_fx(1.635),wl_gx_o.set_fx(-1.635)]
         ).set_intermediate_transverse_beam(False),

    Tier(base=base_point_of_first_portal.shift_y(17.5),
         tier_type= TierType.Flare,
         wind_loads = [wl_gx.set_fx(2.549),wl_gx_o.set_fx(-2.549)]
         ).set_intermediate_transverse_beam(False),
]

tiers = sorted(tiers,key = lambda tier_x : tier_x.base.y)
tier_elevations = [tier.base.y for tier in tiers]
tier_dict = { tier.base.y : tier for tier in tiers }

flares = [
    Flare(lines=[Line3D(Point3D(2.5,17.5,portal_distances[0]),Point3D(2.5,17.5,portal_distances[-1]))],support_member=True,design_load=3),
    Flare(lines=[Line3D(Point3D(5,17.5,portal_distances[0]),Point3D(5,17.5,portal_distances[-1]))],support_member=False,design_load=0.35),
    Flare(lines=[Line3D(Point3D(6,17.5,portal_distances[0]),Point3D(6,17.5,portal_distances[-1]))],support_member=False,design_load=0.3),
]

walkways = [
    SteelWalkway(edge_line=Line3D(Point3D(1,14.5,portal_distances[0]),Point3D(1,14.5,portal_distances[-1]))),
    SteelWalkway(edge_line=Line3D(Point3D(4.5,14.5,portal_distances[0]),Point3D(4.5,14.5,portal_distances[-1]))),
]

ducts = [
    InstrumentationDuct(width=1.2,height=0.4,edge_line=Line3D(Point3D(6,14.5,portal_distances[0]),Point3D(6,14.5,portal_distances[-1])))
]

electric_trees = [
    TreeSupportMember(line=Line3D(Point3D(3,14.5,portal_distances[0]),Point3D(3,14.5,portal_distances[-1])),)
]

ww_support_members = [line for ww_x in walkways for line in ww_x.get_member_lines()]
flare_support_members = [line for flare in flares if flare.support_member for line in flare.lines]
duct_support_members = [line for duct_x in ducts for line in duct_x.get_member_lines()]
tree_support_members = [et.line for et in electric_trees if et.support_member]

long_beam_elevations = [*[tiers[tier_i].base.mid(tiers[tier_i+1].base).y for tier_i in range(len(tiers)-1)],tier_elevations[-1]]
long_beam_elevations_set = set(long_beam_elevations)

pedestal_height = 2
foundation_depth = 1

#portals 
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []
plan_braces = []
vertical_braces = []
portal_beam_ids = [] 
portal_column_ids = [] 
portal_pedestal_ids = [] 
support_node_ids = []
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []
plan_brace_ids = []
vertical_brace_ids = []
portal_count = len(portal_distances)

# Create first portal
portal : PiperackPortal = PiperackPortal(base=base_point_of_first_portal) 

column_distances_set = set(column_distances)

for z_column in column_distances:
    support_point = Point3D(z_column,0,0)-Point3D(0,pedestal_height,0)
    portal.add_pedestal(Column3D(base=support_point,height=pedestal_height))
    portal.add_column(Column3D(base=Point3D(z_column,0,0),height=tier_elevations[-1])) 
    
for tier_x in tier_elevations:
    for column_x in range(len(column_distances)-1):
        portal.add_beam(Beam3D(start=Point3D(column_distances[column_x],tier_x,0),end=Point3D(column_distances[column_x+1],tier_x,0))) 

# Create all portals by shifting the first portal
portals = [portal.shift(Point3D(0,0,dist)) for dist in portal_distances]

# Create longitudinal beams between portals
for portal_i in range(len(portals)-1):
    for z_column in column_distances:
        for long_beam in long_beam_elevations:
            long_beam_x = Beam3D(start=Point3D(z_column,long_beam,portals[portal_i].base.z),end=Point3D(z_column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)

max_portal_to_portal = max([portals[portal_i].base.distance_to(portals[portal_i+1].base) for portal_i in range(len(portals)-1)])

portal_zs = [portal.base.z for portal in portals]
portal_z_set = set(portal_zs)

nodes:dict = get_node_incidences(geometry=geometry)
beam_objects = get_beam_objects(geometry=geometry,property=None,nodes=nodes)
beam_nos:list = list(beam_objects.keys())

tier_elevations = [tier.base.y for tier in tiers]
portal_zs = [portal.base.z for portal in portals]

steel_members = [beam for beam in beam_objects.values() if beam.start.y >= base_point_of_first_portal.y]
concrete_members = [beam for beam in beam_objects.values() if beam.start.y < base_point_of_first_portal.y]

columns = [beam for beam in steel_members if (not beam.start.eq_y(beam.end) and beam.start.eq_x(beam.end) and beam.start.eq_z(beam.end))]
columns_y = group_beams_by_y(columns)
columns_z = group_beams_by_z(columns)
vertical_braces = [beam for beam in beam_objects.values() if (not beam.start.eq_y(beam.end) and beam.start.eq_x(beam.end) and not beam.start.eq_z(beam.end))]
plan_braces = [beam for beam in beam_objects.values() if (beam.start.eq_y(beam.end) and not beam.start.eq_x(beam.end) and not beam.start.eq_z(beam.end))]
long_beams = [beam for beam in beam_objects.values() if (
    beam.start.eq_x(beam.end) and 
    beam.start.eq_y(beam.end) and 
    not beam.start.eq_z(beam.end) and 
    (beam.start.y in long_beam_elevations_set) and
    (beam.start.x in column_distances_set)
)]

int_long_beams = [beam for beam in beam_objects.values() if (
    beam.start.eq_x(beam.end) and 
    beam.start.eq_y(beam.end) and 
    not beam.start.eq_z(beam.end) and 
    (beam.start.y in set(tier_elevations)) and
    not (beam.start.x in column_distances_set)
)]

portal_beams = {}
intermediate_transverse_beams = {}

portal_beam_dict = {portal.base.z:[] for portal in portals}
portal_tier_beams = {portal:{tier:[] for tier in tier_elevations} for portal in portal_zs}

for beam_no,beam in beam_objects.items():
    
    if(portal_zs[0] <= beam.start.z <= portal_zs[-1] and portal_zs[0] <= beam.end.z <= portal_zs[-1]):
        tier_y = None
        portal_z = None
        chosen_one_1,chosen_one_2 = False,False
        
        beam:Beam3D
        
        for tier_x in tier_elevations:
            if(beam.start.eq_z(beam.end) and beam.start.eq_y(tier_x) and beam.end.eq_y(tier_x)):
                chosen_one_1 = True
                tier_y = tier_x
        
        for portal in portal_zs:
            if(beam.start.eq_z(portal) and beam.end.eq_z(portal)):
                chosen_one_2 = True
                portal_z = portal
        
        if(chosen_one_1 and chosen_one_2):
            portal_beams[beam_no] = beam
            if(tier_y and tier_x in tier_dict):
                tier_dict[tier_y].add_beam(beam)
            if(portal_z and portal_z in portal_beam_dict):
                portal_beam_dict[portal_z].append(beam)
                
            portal_tier_beams[portal_z][tier_y].append(beam_no)
            
        if(chosen_one_1 and not chosen_one_2):
            intermediate_transverse_beams[beam_no] = beam
            if(tier_y in tier_dict):
                tier_dict[tier_y].add_int_beam(beam)


display(HTML(dark_theme_css))
display(widgets.HBox([
    create_button_bar_for_dict(heading='Columns', item_dict=columns_y, selector=select_beams),
    create_button_bar_for_dict(heading='Tier Beams', item_dict={tier.base.y: tier.beams for tier in tiers}, selector=select_beams),
    create_button_bar_for_dict(heading='Tier Int Beams', item_dict={tier.base.y: tier.int_beams for tier in tiers}, selector=select_beams),
    create_button_bar_for_dict(heading='VB', item_dict=group_beams_by_y(vertical_braces), selector=select_beams),
    create_button_bar_for_dict(heading='PB', item_dict=group_beams_by_y(plan_braces), selector=select_beams),
    create_button_bar_for_dict(heading='Long Beams', item_dict=group_beams_by_y(long_beams), selector=select_beams),
    create_button_bar_for_dict(heading='Long Int Beams', item_dict=group_beams_by_y(int_long_beams), selector=select_beams)
], layout=dark_hbox))

staad_profiles = get_staad_profiles()
staad_section_ref_nos = {profile['name']:profile for profile in staad_profiles}

staad_profiles_widget = display_data_widget(staad_profiles)
steel_widget, steel_dropdown = create_steel_section_widget(steel_sections)

profile_button_bar = widgets.VBox([create_button(label='Insert Profile',predicate=lambda:insert_profile_button_click(get_section_ref_no=get_section_ref_no,
                                                                                                                     steel_dropdown=steel_dropdown,
                                                                                                                     staad_section_ref_nos=staad_section_ref_nos,
                                                                                                                     )), 
                                  
                                   create_button(label='Apply Profile',predicate=lambda:apply_profile_button_click(get_section_ref_no=get_section_ref_no,
                                                                                                                   steel_dropdown=steel_dropdown,
                                                                                                                   staad_section_ref_nos=staad_section_ref_nos,
                                                                                                                   geometry=geometry,get_selected_beam_nos=get_selected_beam_nos,
                                                                                                                   beam_list_copy_and_display=beam_list_copy_and_display,
                                                                                                                   assign_profile=assign_profile))])

display(widgets.HBox([staad_profiles_widget,steel_widget, profile_button_bar]))